# 项目： 基于评论的情感分析

本项目的目标是基于用户提供的评论，通过算法自动去判断其评论是正面的还是负面的情感。比如给定一个用户的评论：
- 评论1： “我特别喜欢这个电器，我已经用了3个月，一点问题都没有！”
- 评论2： “我从这家淘宝店卖的东西不到一周就开始坏掉了，强烈建议不要买，真实浪费钱”

对于这两个评论，第一个明显是正面的，第二个是负面的。 我们希望搭建一个AI算法能够自动帮我们识别出评论是正面还是负面。

情感分析的应用场景非常丰富，也是NLP技术在不同场景中落地的典范。比如对于一个证券领域，作为股民，其实比较关注舆论的变化，这个时候如果能有一个AI算法自动给网络上的舆论做正负面判断，然后把所有相关的结论再整合，这样我们可以根据这些大众的舆论，辅助做买卖的决策。 另外，在电商领域评论无处不在，而且评论已经成为影响用户购买决策的非常重要的因素，所以如果AI系统能够自动分析其情感，则后续可以做很多有意思的应用。 

情感分析是文本处理领域经典的问题。整个系统一般会包括几个模块：
- 数据的抓取： 通过爬虫的技术去网络抓取相关文本数据
- 数据的清洗/预处理：在本文中一般需要去掉无用的信息，比如各种标签（HTML标签），标点符号，停用词等等
- 把文本信息转换成向量： 这也成为特征工程，文本本身是不能作为模型的输入，只有数字（比如向量）才能成为模型的输入。所以进入模型之前，任何的信号都需要转换成模型可识别的数字信号（数字，向量，矩阵，张量...)
- 选择合适的模型以及合适的评估方法。 对于情感分析来说，这是二分类问题（或者三分类：正面，负面，中性），所以需要采用分类算法比如逻辑回归，朴素贝叶斯，神经网络，SVM等等。另外，我们需要选择合适的评估方法，比如对于一个应用，我们是关注准确率呢，还是关注召回率呢？ 这跟应用场景以及数据本身有关。比如训练数据里，有100个正样本，1个负样本，那这时候就不太适合用准确率来衡量系统的好坏，为什么？ 因为在这种样本很不均匀的情况下，我直接把所有的数据分类成正样本，那在没有任何学习的情况下准确率也可以达到100/101，接近100%，这显然是不太合理的。所以在这种情况我们更趋向于用其他的评估方式比如AUC。评估方式是影响系统的关键因素，因为任何的学习过程都是在不断地优化我们制定的评估方式。

在本次项目中，我们已经给定了训练数据和测试数据，它们分别是 train.positive.txt, train.negative.txt， test_combined.txt. 请注意训练数据和测试数据的格式不一样，详情请见文件内容。 整个项目你需要完成以下步骤：
- 数据的读取以及清洗： 从给定的.txt中读取内容，并做一些数据清洗，这里需要做几个工作： （1） 文本的读取，需要把字符串内容读进来。 （2）去掉无用的字符比如标点符号，多余的空格，换行符等  （3） 分词 
- 把文本转换成TF-IDF向量： 这部分直接可以利用sklearn提供的TfidfVectorizer类来做。
- 利用逻辑回归模型来做分类，并通过交叉验证选择最合适的超参数
- 利用神经网络，支持向量机做分类，并通过交叉验证选择神经网络的合适的参数

### 第一部分： 数据预处理
本部分你将要完成数据的预处理过程，包括数据的读取，数据清洗，分词，以及把文本转换成tf-idf向量。请注意: 在接下来的任务中，正面的情感我们标记为1， 负面
的情感我们标记成0

In [1]:
import jieba #引入分词库jieba
def process_line(line):
    """
    给定一行文字，进行清洗和分词操作。
    输入：“人工智能怎么学啊？？？， 为什么总是学不好。。。，！ 是不是学习方法有问题呀 zzzzz”
    输出：”人工智能 怎么 学 啊 为什么 总是 学 不好 是不是 学习 方法 有 问题 呀“
    """
    # TODO step 1: 去掉文字里面的标点符号（比如 ? # ! ...) ，和多余的空格，保证分词后词与词之间只有一个空格
    for ch in "~!@#$%^&*()_+·`，～（）。、；“！ ．？”‘’＂【…】《》1234567890QWERTYUIOPASDFGHJKLZXCVBNMqwertyuiopasdfghjklzxcvbnm-={}[]\|;':<>,./?":
        line = line.replace(ch,"").replace('\n','').replace('"','')
    for ch in "的是吗啦呀啊哦嗯嘛了":
        line = line.replace(ch,"")
    # TODO step 2: 用jieba分词 请参考：https://github.com/fxsjy/jieba, 必要时调整一下jieba的词典库
    line = jieba.lcut(line)
    new_line = " ".join(line) #list转str
    return new_line

In [2]:
import numpy as np
import re #引入正则表达式模块
def process_train(file_path):
    """
    处理训练数据， “train_positive.txt” OR "train_negative.txt"。
    主要的工作是提取出每一行评论文本，并对提取出来的每一行文本通过process_line函数来做处理
    """
    comments = []     # 空列表：用来存储评论
    if   file_path =='train.positive.txt':
        lis = [1]
    elif file_path =='train.negative.txt':
        lis = [0]
    labels = lis*5000 # 用来存储标签（正/负），如果是train_positive.txt，则所有标签为1， 否则0. 

    with open(file_path,'r', encoding='utf-8') as file:
        # TODO 提取每一个评论，然后利用process_line函数来做处理，并添加到comments。
        a = re.findall(r"<review id=(.+?)</review>",file.read(), re.S) #使用正则表达式匹配文档中的的每一条有用信息
        for i in a:
            line = "".join(i) #list转str
            line = process_line(line)
            comments.append(line) #将处理好的文本(若干条str)添加到comments
    return comments, labels

In [3]:
def process_test(file_path):
    """
    处理测试数据， “test_combined.txt"
    主要的工作是提取出每一行评论文本，需要做简单的解析。test_combined.txt里面既包含了正面的也包含了负面的，需要分别
    提取出来，并使用process_line函数来处理，最后放到指定的List里面再返回
    """
    comments = []  # 用来存储评论
    labels = []    # 用来存储标签(正/负).
    with open(file_path,'r', encoding='utf-8') as file:
        # TODO 提取每一个评论，然后利用process_line函数来做处理，并添加到
        # comments列表里，并添加指定的标签（正/负）到labels列表里。
        b = re.findall(r"<review id=(.+?)</review>",file.read(), re.S)
        labels = re.findall(r'label="(.+?)',"".join(b))#获取标签中的labels，转str
        labels = list(map(int, labels))  #去掉labels里面的单引号 ['1', '0', '1']-->[1, 0, 1]，其实是str转list
        for i in b:
            line = "".join(i)
            line = process_line(line)
            comments.append(line)
    return comments, labels

In [4]:
def read_file():
    """
    读取所提供的.txt文件，并把内容处理之后写到list里面。 这里需要分别处理四个文件，“train_positive.txt", "train_negative.txt",
    "test_combined.txt" 并把每一个文件里的内容存储成列表。 
    """
    # 处理训练数据，这两个文件的格式相同，请指定训练文件的路径
    train_pos_comments, train_pos_labels = process_train("train.positive.txt")
    train_neg_comments, train_neg_labels = process_train("train.negative.txt")
    
    # TODO: train_pos_comments和train_neg_comments合并成train_comments， train_pos_labels和train_neg_labels合并成train_labels
    train_comments = train_pos_comments+train_neg_comments
    train_labels = train_pos_labels+train_neg_labels
    
    # 处理测试数据, 请指定测试文件的路径
    test_comments, test_labels = process_test("test.combined.txt")
    
    return train_comments, train_labels, test_comments, test_labels


In [5]:
# 读取数据，并对文本进行处理
train_comments, train_labels, test_comments, test_labels = read_file()

# 查看训练数据与测试数据大小
print (len(train_comments), len(train_labels), len(test_comments), len(test_labels))
print(type(train_comments), type(train_labels), type(test_comments), type(test_labels))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Leo\AppData\Local\Temp\jieba.cache
Loading model cost 0.991 seconds.
Prefix dict has been built succesfully.


10000 10000 2500 2500
<class 'list'> <class 'list'> <class 'list'> <class 'list'>


In [8]:
# 把每一个文本内容转换成tf-idf向量
from sklearn.feature_extraction.text import TfidfVectorizer  # 导入sklearn库
# TODO: 利用TfidfVectorizer把train_comments转换成tf-idf，把结果存储在X_train, 这里X_train是稀疏矩阵（Sparse Matrix） 
# 并把train_labels转换成向量 y_train. 类似的，去创建X_test, y_test。 把文本转换成tf-idf过程请参考TfidfVectorizer的说明

from sklearn import linear_model
vectorizer = TfidfVectorizer(max_features=10000) #max_features=10000：设定词向量长度
X_train = vectorizer.fit_transform(train_comments) #训练数据集使用‘.fit_transform’
X_test  = vectorizer.transform(test_comments)   #测试数据集使用‘.transform’（当不指定max_features时，测试数据集长度将默认等于训练集）

y_train = np.array(train_labels).T #把上面的词向量转换成稀疏矩阵
y_test  = np.array(test_labels).T
# 查看每个矩阵，向量的大小, 保证X_train和y_train, X_test和y_test的长度是一样的。
print (np.shape(X_train), np.shape(y_train), np.shape(X_test), np.shape(y_test))

(10000, 10000) (10000,) (2500, 10000) (2500,)


### 第二部分： 利用逻辑回归模型搭建情感分析引擎
在本部分你将会利用罗回归模型（logistic regressiion）来搭建情感分析引擎。你需要完成整个pipeline的搭建过程，从而学会一个机器学习模型的搭建流程。 

In [9]:
from sklearn.linear_model import LogisticRegression
# TODO： 初始化模型model，并利用模型的fit函数来做训练，暂时用默认的设置。
# 具体使用方法请参考：http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
model_lr = LogisticRegression()
model_lr.fit(X_train, y_train)

# 打印在训练数据上的准确率
print ("训练数据上的准确率为：" + str(model_lr.score(X_train, y_train)))

# 打印在测试数据上的准确率
print ("测试数据上的准确率为: " + str(model_lr.score(X_test, y_test)))

# TODO： 打印混淆矩阵（confusion matrix）。
# 参考：http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
# 混淆矩阵是一种常用的分析方法。比如模型的准确率不理想的情况下，可以做进一步地分析，并找出原因。 在情感分析问题上，
# 混淆矩阵可以用来分析有多少个原本正样本被分类成负样本，有多少原本是负样本的被分类成正样本，可以做这种精细化的结果分析，从而找到一些原因。 

# TODO: 利用自己提出的例子来做测试。随意指定一个评论，接着利用process_line来做预处理，再利用之前构建好的TfidfVectorizer来把文本转换
# 成tf-idf向量， 然后再利用构建好的model做预测（model.predict函数）
test_comment1 = vectorizer.transform(["买到赚到，这玩意太值了！！！推荐！"])
test_comment2 = vectorizer.transform(["还不错！！但是aaa淘@#$%^&*(宝有更便宜的~"])
test_comment3 = vectorizer.transform(["什么玩意啊？？太坑了吧，还卖这么贵！！"])
# TODO: 输出结果，并自己分析一下是不是跟自己想象的结果一致。 也可以输出两个分类的概率
print(model_lr.predict(test_comment1),model_lr.predict(test_comment2),model_lr.predict(test_comment3))

训练数据上的准确率为：0.8479
测试数据上的准确率为: 0.7648
[1] [0] [0]


D:\Program_files\Anaconda\ANACONDA-2019\envs\TF-CPU3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


### 第三部分： 利用决策树，神经网络，SVM来训练模型。 
本部分类似于第二部分的内容，只不过替换成其他的模型（包括决策树，神经网络，SVM模型）

In [10]:
# 利用决策树来做情感分析预测
from sklearn import tree
# 具体使用方法请参考：http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

# TODO: 初始化决策树模型，并利用模型的fit函数来做训练并打印在训练和测试数据上的准确率，利用决策树默认的参数设置
model_tree1 = tree.DecisionTreeClassifier().fit(X_train,y_train)
# 打印在训练数据上的准确率
print ("Tree1训练数据上的准确率为：" + str(model_tree1.score(X_train, y_train)))

# 打印在测试数据上的准确率
print ("Tree1测试数据上的准确率为: " + str(model_tree1.score(X_test, y_test)))


# TODO: 初始化决策树模型，并利用模型的fit函数来做训练并打印在训练和测试数据上的准确率，设置max_depth参数为3
model_tree2 = tree.DecisionTreeClassifier(max_depth=3).fit(X_train,y_train)
# 打印在训练数据上的准确率
print ("Tree2训练数据上的准确率为：" + str(model_tree2.score(X_train, y_train)))

# 打印在测试数据上的准确率
print ("Tree2测试数据上的准确率为: " + str(model_tree2.score(X_test, y_test)))


# TODO: 初始化决策树模型，并利用模型的fit函数来做训练并打印在训练和测试数据上的准确率，设置max_depth参数为5
model_tree3 = tree.DecisionTreeClassifier(max_depth=5).fit(X_train,y_train)
# 打印在训练数据上的准确率
print ("Tree3训练数据上的准确率为：" + str(model_tree3.score(X_train, y_train)))

# 打印在测试数据上的准确率
print ("Tree3测试数据上的准确率为: " + str(model_tree3.score(X_test, y_test)))

Tree1训练数据上的准确率为：0.9909
Tree1测试数据上的准确率为: 0.6724
Tree2训练数据上的准确率为：0.6069
Tree2测试数据上的准确率为: 0.6044
Tree3训练数据上的准确率为：0.6242
Tree3测试数据上的准确率为: 0.6136


In [11]:
# 利用支持向量机（SVM）来做情感分析预测
from sklearn import svm
# http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

# TODO: 初始化SVM模型，并利用模型的fit函数来做训练并打印在训练和测试数据上的准确率，SVM模型的kernel设置成“rbf”核函数
model_svm = svm.SVC(kernel='rbf').fit(X_train,y_train)
# 打印在训练数据上的准确率
print ("SVM训练数据上的准确率为：" + str(model_svm.score(X_train, y_train)))

# 打印在测试数据上的准确率
print ("SVM测试数据上的准确率为: " + str(model_svm.score(X_test, y_test)))

D:\Program_files\Anaconda\ANACONDA-2019\envs\TF-CPU3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVM训练数据上的准确率为：0.5188
SVM测试数据上的准确率为: 0.5148


思考题：“通常情况下SVM模型的训练效率要远低于逻辑回归，决策树或者其他的模型，为什么？“ (难度：四星)

In [12]:
# 利用线性支持向量机(LinearSVM)来做情感分析的预测
from sklearn.svm import LinearSVC
# 具体的使用方式请见： http://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html

# TODO: 初始化LinearSVC模型，并利用模型的fit函数来做训练并打印在训练和测试数据上的准确率，使用模型的默认参数。
model_lsvm = LinearSVC().fit(X_train,y_train)
# 打印在训练数据上的准确率
print ("LinearSVM训练数据上的准确率为：" + str(model_lsvm.score(X_train, y_train)))

# 打印在测试数据上的准确率
print ("LinearSVM测试数据上的准确率为: " + str(model_lsvm.score(X_test, y_test)))

LinearSVM训练数据上的准确率为：0.9268
LinearSVM测试数据上的准确率为: 0.7384


思考题：“同样是SVM模型，但会发现后者的训练速度要远快于前者，请思考一下为什么？” (难度：五星)

In [11]:
# 利用神经网络模型来做情感分析的预测
from sklearn.neural_network import MLPClassifier
# 具体使用方法请见：http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html

# TODO: 初始化MLPClassifier模型，并利用模型的fit函数来做训练并打印在训练和测试数据上的准确率，设置为hidden_layer_sizes为100，
# 并使用"lbfgs" solver
model_nn = MLPClassifier(hidden_layer_sizes=(100),solver='lbfgs',random_state=1).fit(X_train,y_train)
# 打印在训练数据上的准确率
print ("NN训练数据上的准确率为：" + str(model_nn.score(X_train, y_train)))

# 打印在测试数据上的准确率
print ("NN测试数据上的准确率为: " + str(model_nn.score(X_test, y_test)))

NN训练数据上的准确率为：0.9885
NN测试数据上的准确率为: 0.7092


### 第四部分： 通过交叉验证找出最好的超参数
调用一个sklearn模型本身很简单，只需要2行代码即可以完成所需要的操作。但这里的关键点在于怎么去寻找最优的超参数（hyperparameter）。 比如对于逻辑回归
来说，我们可以设定一些参数的值如“penalty”, C等等，这些我们可以理解成是超参数。通常情况下，超参数对于整个模型的效果有着举足轻重的作用，这就意味着
我们需要一种方式起来找到一个比较合适的参数。其中一个最常用的方法是grid search, 也就在一个去区间里面做搜索，然后找到最优的那个参数值。 

举个例子，对于逻辑回归模型，它拥有一个超参数叫做C,在文档里面解释叫做“Inverse of regularization strength“， 就是正则的权重，而且这种权重的取值
范围可以认为通常是（0.01, 1000）区间。这时候，通过grid search的方式我们依次可以尝试 0.01, 0.1, 1, 10, 100, 1000 这些值，然后找出使得
模型的准确率最高的参数。当然，如果计算条件资源允许的话，可以尝试更多的值，比如0.01,0.05,0.1, 0.5, 1, 5, 10 ..。 当我们尝试越多值的时候，找到
最优参数的概率就会越大。

另外，参数的搜索过程离不开交叉验证，交叉验证相关的细节请参考线上的视频课程。

在第四部分里，你将要编写程序来寻找最优的参数，分别针对逻辑回归和神经网络模型。

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
params_c = np.logspace(-3,3,7)   # 对于参数 “C”，尝试几个不同的值
best_c = params_c[0]  # 存储最好的C值
best_acc = 0
for c in params_c:
    # TODO： 编写交叉验证的过程，对于每一个c值，计算出在验证集中的平均准确率。 在这里，我们做5-fold交叉验证。也就是，每一次把20%
    #   的数据作为验证集来对待，然后准确率为五次的平均值。我们把这个准确率命名为 acc_avg
    model_LR = LogisticRegression(solver='liblinear',C=c)
    kfold = StratifiedKFold(n_splits=5,shuffle=True,random_state=1) #交叉验证
    acc_avg = cross_val_score(model_LR,X_train, y_train,cv=kfold).mean()
    if acc_avg > best_acc:
        best_acc = acc_avg
        best_c = c
    
print ("最好的参数C值为： %f" % (best_c))
# TODO 我们需要在整个训练数据上重新训练模型，但这次利用最好的参数best_c值
#     提示： model = LogisticRegression(C=best_c).fit(X_train, y_train)
model_LR = LogisticRegression(solver='liblinear',C=best_c)
model_LR.fit(X_train, y_train)

# 打印在训练数据上的准确率
print ("训练数据上的准确率为：" + str(model_LR.score(X_train, y_train)))

# 打印在测试数据上的准确率
print ("测试数据上的准确率为: " + str(model_LR.score(X_test, y_test)))

最好的参数C值为： 1.000000
训练数据上的准确率为：0.8479
测试数据上的准确率为: 0.7648


In [13]:
from sklearn.neural_network import MLPClassifier
param_hidden_layer_sizes = np.linspace(50, 200, 11)  # 针对参数 “hidden_layer_sizes”, 尝试几个不同的值
param_alphas = np.logspace(-4,1,6)  # 对于参数 "alpha", 尝试几个不同的值

best_hidden_layer_size = param_hidden_layer_sizes[0]
best_alpha = param_alphas[0]
best_acc = 0

In [25]:
#交叉验证（比较耗时间，下面代码运行需要约2h）
for size in param_hidden_layer_sizes:
    for val in param_alphas:
        # TODO 编写交叉验证的过程，需要做5-fold交叉验证。
        model_NN = MLPClassifier(hidden_layer_sizes=int(size),alpha=val,solver='lbfgs',random_state=1)
        kfold = StratifiedKFold(n_splits=5,shuffle=True,random_state=1) #交叉验证
        acc_avg = cross_val_score(model_NN,X_train, y_train,cv=kfold).mean()
        if acc_avg > best_acc:
            best_acc = acc_avg
            best_hidden_layer_size = size
            best_alpha = val

print ("最好的参数hidden_layer_size值为： %f" % (best_hidden_layer_size))
print ("最好的参数alpha值为： %f" % (best_alpha))

# TODO 我们需要在整个训练数据上重新训练模型，但这次使用最好的参数hidden_layer_size和best_alpha
model_NN = MLPClassifier(hidden_layer_sizes=int(best_hidden_layer_size),alpha=best_alpha,solver='lbfgs',random_state=1)
model_NN.fit(X_train,y_train)

# 打印在训练数据上的准确率
print ("训练数据上的准确率为：" + str(model_NN.score(X_train, y_train)))

# 打印在测试数据上的准确率
print ("测试数据上的准确率为: " + str(model_NN.score(X_test, y_test)))  

最好的参数hidden_layer_size值为： 95.000000
最好的参数alpha值为： 10.000000
训练数据上的准确率为：0.9372
测试数据上的准确率为: 0.7388
